In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from PIL import Image
import os
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import Normalize
import math

### create low res copy of all test images

In [4]:
def downsample_image(image_path, output_size):
    # Open the image
    image = Image.open(image_path)

    # Perform downsampling using bicubic interpolation
    downscaled_image = image.resize(output_size, resample=Image.BICUBIC)

    return downscaled_image

# Folder path containing the images
folder_path = "/kaggle/input/animal-faces/afhq/val/dog"

# Defining folder for downscaled images serving for input for modelling (&upscaling)
output_folder_path = "/kaggle/working/dog_low_res"
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)


# Output size for downsampling (by a factor of 3)
output_size = (128, 128)

# Iterate over the files in the folder
for number, filename in enumerate(os.listdir(folder_path)):

  # Construct the full path to the image file
  image_path = os.path.join(folder_path, filename)

  # Apply downsampling to the image
  downsampled_image = downsample_image(image_path, output_size)

  # Save the downscaled image
  output_filename = f"downsampled_{filename}"
  output_path = os.path.join(output_folder_path, output_filename)
  downsampled_image.save(output_path)


### using same data loaders built for training process

In [5]:
low_res_folder = "./data/Human/Humans_final_200_test"
high_res_folder = "./data/Human/Humans_final_600_test"


# === creating dataset with all images ===
class CustomDataset(Dataset):
    def __init__(self, low_res_folder, high_res_folder, transform=None):
        self.low_res_folder = low_res_folder
        self.high_res_folder = high_res_folder
        self.low_res_images = sorted(os.listdir(low_res_folder))
        self.high_res_images = sorted(os.listdir(high_res_folder))
        self.transform = transform

    def __len__(self):
        return len(self.low_res_images)

    def __getitem__(self, index):
        low_res_image = Image.open(os.path.join(self.low_res_folder, self.low_res_images[index]))
        high_res_image = Image.open(os.path.join(self.high_res_folder, self.high_res_images[index]))

        if self.transform is not None:
            low_res_image = self.transform(low_res_image)
            high_res_image = self.transform(high_res_image)

        return low_res_image, high_res_image

# transform to tensor & normalize
normalize = Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])

base_transform = transforms.Compose([
    transforms.ToTensor()
])

# original dataset
dataset = CustomDataset(low_res_folder, high_res_folder, transform=base_transform)

batch_size = 64
test_loader = DataLoader(dataset, batch_size=batch_size)

### defining structure of all models (needed since saving a model only saves the weights)

In [6]:
class bicubic(nn.Module):
    def __init__(self):
        super(bicubic, self).__init__()
        self.interpolation = nn.Upsample(scale_factor=4, mode='bicubic')

    def forward(self, x):
        x = self.interpolation(x)
        return x


In [7]:
# SRCNN model
class SRCNN(nn.Module):
    def __init__(self):
        super(SRCNN, self).__init__()
        self.interpolation = nn.Upsample(scale_factor=4, mode='bicubic')
        self.conv1 = nn.Conv2d(3, 64, kernel_size=9, stride=1, padding=4)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(64, 32, kernel_size=1, stride=1, padding=0)
        self.relu2 = nn.ReLU()
        self.conv3 = nn.Conv2d(32, 3, kernel_size=5, stride=1, padding=2)
        self.relu3 = nn.ReLU()

    def forward(self, x):
        x = self.interpolation(x)
        x = self.relu1(self.conv1(x))
        x = self.relu2(self.conv2(x))
        x = self.relu3(self.conv3(x))
        return x

In [2]:
# Load the trained model
class FSRCNN(nn.Module):
    def __init__(self, d=88, s=18, m=6):
        super(FSRCNN, self).__init__()
        # Feature Extraction
        self.conv1 = nn.Conv2d(3, d, kernel_size=5, padding=2)
        self.relu1 = nn.PReLU(d)
        # Shrinking
        self.conv2 = nn.Conv2d(d, s, kernel_size=1)
        self.relu2 = nn.PReLU(s)
        # Non-linear Mapping
        self.mapping = nn.Sequential(*[nn.Sequential(
            nn.Conv2d(s, s, kernel_size=3, padding=1),
            nn.PReLU(s)
        ) for _ in range(m)])
        # Expanding
        self.conv3 = nn.Conv2d(s, d, kernel_size=1)
        self.relu3 = nn.PReLU(d)
        # Deconvolution
        self.deconv = nn.ConvTranspose2d(d, 3, kernel_size=9, stride=4, padding=4, output_padding=3)

    def forward(self, x):
        x = self.relu1(self.conv1(x))
        x = self.relu2(self.conv2(x))
        x = self.mapping(x)
        x = self.relu3(self.conv3(x))
        x = self.deconv(x)
        return x

In [ ]:
class ESPCN(nn.Module):
    def __init__(self, upscale_factor=4, num_channels=3):
        super(ESPCN, self).__init__()
        self.upscale_factor = upscale_factor
        
        # Feature Extraction
        self.conv1 = nn.Conv2d(num_channels, 64, kernel_size=5, padding=2)
        self.relu1 = nn.ReLU()
        
        # Sub-Pixel Convolution
        self.conv2 = nn.Conv2d(64, num_channels * upscale_factor ** 2, kernel_size=3, padding=1)
        self.pixel_shuffle = nn.PixelShuffle(upscale_factor)
        self.relu2 = nn.ReLU()
    
    def forward(self, x):
        x = self.relu1(self.conv1(x))
        x = self.pixel_shuffle(self.conv2(x))
        x = self.relu2(x)
        return x


### loading a saved model (only run the cell of one modell at a time here)

In [14]:
model = bicubic()
model.load_state_dict(torch.load("/kaggle/input/bicubic/bicubic.pth", map_location=torch.device('cpu')))
model.eval()

bicubic(
  (interpolation): Upsample(scale_factor=4.0, mode='bicubic')
)

In [5]:
model = FSRCNN()
model.load_state_dict(torch.load("./Modelle/Human/FSRCNN_20.pth", map_location=torch.device('cpu')))
model.eval()

FSRCNN(
  (conv1): Conv2d(3, 88, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu1): PReLU(num_parameters=88)
  (conv2): Conv2d(88, 18, kernel_size=(1, 1), stride=(1, 1))
  (relu2): PReLU(num_parameters=18)
  (mapping): Sequential(
    (0): Sequential(
      (0): Conv2d(18, 18, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): PReLU(num_parameters=18)
    )
    (1): Sequential(
      (0): Conv2d(18, 18, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): PReLU(num_parameters=18)
    )
    (2): Sequential(
      (0): Conv2d(18, 18, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): PReLU(num_parameters=18)
    )
    (3): Sequential(
      (0): Conv2d(18, 18, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): PReLU(num_parameters=18)
    )
    (4): Sequential(
      (0): Conv2d(18, 18, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): PReLU(num_parameters=18)
    )
    (5): Sequential(
      (0): Conv2d(18, 18, kernel_size

### test the model on a single defined image

In [6]:
# Preprocess the test image
path = "./data/Human/Humans_final_200_test/"
filename = "downsampled_downsampled_1 (6898).jpg"
test_image_path = path+filename
test_image = Image.open(test_image_path)
transform = transforms.Compose([
    transforms.ToTensor()
])
test_image = transform(test_image).unsqueeze(0)

# Apply the model to the test image
with torch.no_grad():
    output_image = model(test_image)

# Convert the output tensor to an image
output_image = output_image.squeeze(0)
output_image = transforms.ToPILImage()(output_image)

# Save the output image
output_image.save(f"./data/Human/Up/upscaled_{filename}")


### apply the model to all the tesing data (saves upscaled images and calculates PSNR)

In [12]:
test_loss = 0
number_batches = 0
criterion = nn.MSELoss()
device = torch.device("cpu")

img_comparison_folder = "/kaggle/working/img_comparison"

if not os.path.exists(img_comparison_folder):
    os.makedirs(img_comparison_folder)

for input_data, desired_data in test_loader:
    with torch.no_grad():
        number_batches += 1
        if not os.path.exists(f"/kaggle/working/img_comparison/batch_{number_batches}"):
            os.makedirs(f"/kaggle/working/img_comparison/batch_{number_batches}")

        # Move input and desired images to device
        input_data = input_data.to(device)
        desired_data = desired_data.to(device)

        # Forward pass
        output_images = model(input_data)

        # Calculate loss
        loss = criterion(output_images, desired_data)
        test_loss += loss.item()
        
        # saving the images
        for number, output_image in enumerate(output_images):
            output_image = output_image.squeeze(0)
            output_image = transforms.ToPILImage()(output_image)
            output_image.save(f"/kaggle/working/img_comparison/batch_{number_batches}/{number}_upscaled.jpg")
            
        # saving low res images from the same batch into the same folder
        for number, image in enumerate(input_data):
            image = image.squeeze(0)
            image = transforms.ToPILImage()(image)
            image.save(f"/kaggle/working/img_comparison/batch_{number_batches}/{number}_input.jpg")
            
        # saving original images from the same batch into the same folder
        for number, image in enumerate(desired_data):
            image = image.squeeze(0)
            image = transforms.ToPILImage()(image)
            image.save(f"/kaggle/working/img_comparison/batch_{number_batches}/{number}_original.jpg")

test_loss_avg = test_loss / number_batches

# Print training loss per epoch
psnr = 10 * math.log10(1 / test_loss_avg)

print(f"Loss: {test_loss_avg:.4f}, PSNR: {psnr}")

Loss: 0.0020, PSNR: 26.887029451771824


### calculate testing PSNR only (no images will be saved)

In [16]:
import time
test_loss = 0
number_batches = 0
criterion = nn.MSELoss()
device = torch.device("cpu")

start_time = time.time()
for input_data, desired_data in test_loader:
    with torch.no_grad():
        number_batches += 1
        print(number_batches)

        # Move input and desired images to device
        input_data = input_data.to(device)
        desired_data = desired_data.to(device)

        # Forward pass
        output_images = model(input_data)

        # Calculate loss
        loss = criterion(output_images, desired_data)
        test_loss += loss.item()

end_time = time.time()  

elapsed_time = end_time - start_time  
print(f"Duration (total): {elapsed_time} seconds, duration per file: {elapsed_time/len(dataset)} seconds")

test_loss_avg = test_loss / number_batches

# Print training loss per epoch
psnr = 10 * math.log10(1 / test_loss_avg)

print(f"Loss: {test_loss_avg:.4f}, PSNR: {psnr}")

1
2
3
4
5
6
7
8
Loss: 0.0020, PSNR: 27.07621104266254
